In [1]:
import pandas as pd
import seaborn as sns
sns.set_theme()
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
# for now naive cleaning
sessions = pd.read_json("data/sessions.jsonl", lines=True).dropna().set_index('timestamp')


## Listen time user/track

In [2]:
listen_events = sessions[sessions['event_type'].isin(['play', 'skip'])]
cache = []
for idx, session in listen_events.groupby('session_id'):
    session = session.reset_index()
    for i in range(len(session) -1):
        curr = session.iloc[i]
        next_v = session.iloc[i+1]
        if curr["track_id"] != next_v["track_id"]:
            continue
        if curr['event_type'] != 'play' or next_v['event_type'] != 'skip':
            continue
        cache.append({
            'timestamp': curr['timestamp'],
            'user_id': curr['user_id'],
            'track_id': curr['track_id'],
            'duration': next_v['timestamp'] - curr['timestamp']
        })
pd.DataFrame(cache)


,timestamp,user_id,track_id,duration
0,2023-09-04 09:00:42.000,101.0,6cy83P2limyzaFSl3OoQe0,0 days 00:04:13.039000
1,2023-10-25 19:17:17.093,101.0,1LeI5P9uyh9HGpoVZnTz9r,0 days 00:00:19.131000
2,2023-10-25 19:24:24.430,101.0,3HlOk3Iy7vkOfROwBbJqfx,0 days 00:02:54.116000
3,2023-11-05 21:29:24.427,101.0,0NDYm1yTwoyHTiIA8bSw2M,0 days 00:02:20.674000
4,2023-01-31 05:02:28.226,102.0,4dF6hP1Jts3iXkUosvG0kg,0 days 00:01:46.824000
...,...,...,...,...
609,2023-09-13 19:20:13.404,150.0,3JfBv2YfwPdflYUGMCeHgt,0 days 00:01:48.574000
610,2023-09-13 19:22:01.978,150.0,4ZvCSzkZVmGLpbvRjZWLzC,0 days 00:01:06.975000
611,2023-09-16 11:21:15.000,150.0,6xa8EuqmwQPVjzhyir7yTU,0 days 00:02:23.642000
612,2023-09-16 11:23:38.642,150.0,2EqGctscFgYXkv2WEJSWoN,0 days 00:01:09.514000


## if skip occurred user/track

In [12]:
sessions[sessions['event_type']=='skip'].groupby(['user_id', 'track_id']).size().reset_index()

,user_id,track_id,0
0,101.0,0NDYm1yTwoyHTiIA8bSw2M,1
1,101.0,1LeI5P9uyh9HGpoVZnTz9r,1
2,101.0,36lkJSDyMEZoWTqtRj8Q8q,1
3,101.0,3HlOk3Iy7vkOfROwBbJqfx,1
4,101.0,3hQT53VbsScHcmOgZZs9I7,1
...,...,...,...
699,150.0,7EUImYZOKeYVF2H1jQT4DU,1
700,150.0,7K4wD19D9CN4UKC4kIR7Cy,1
701,150.0,7ifcplNdIFySpiB8YhKLzT,1
702,150.0,7nAC0JP2NVoYzPtRdR39BS,1


## user session time

In [17]:
sessions.reset_index().groupby(['session_id', 'user_id']).agg(session_start=('timestamp', 'min'), session_end=('timestamp', 'max')).assign(duration=lambda x: x['session_end'] - x['session_start']).reset_index()

,session_id,user_id,session_start,session_end,duration
0,124,101.0,2023-04-24 04:02:43.669,2023-04-24 04:15:41.241,0 days 00:12:57.572000
1,125,101.0,2023-05-25 15:30:22.000,2023-05-25 15:30:22.000,0 days 00:00:00
2,126,101.0,2023-07-18 11:16:23.000,2023-07-18 11:20:15.679,0 days 00:03:52.679000
3,127,101.0,2023-08-23 20:55:13.000,2023-08-23 20:55:13.000,0 days 00:00:00
4,128,101.0,2023-09-04 09:00:42.000,2023-09-04 09:11:08.133,0 days 00:10:26.133000
...,...,...,...,...,...
616,806,150.0,2023-08-31 04:16:45.000,2023-08-31 04:19:50.693,0 days 00:03:05.693000
617,807,150.0,2023-09-05 23:45:58.000,2023-09-05 23:49:03.082,0 days 00:03:05.082000
618,808,150.0,2023-09-13 19:10:41.000,2023-09-13 19:33:35.740,0 days 00:22:54.740000
619,809,150.0,2023-09-16 11:21:15.000,2023-09-16 11:32:14.634,0 days 00:10:59.634000


## skip occurrence with time of day by day

In [20]:
skip_occurrence = sessions[sessions['event_type']=='skip']
skip_occurrence['hour'] = skip_occurrence.index.hour
skip_occurrence['date'] = skip_occurrence.index.date
skip_occurrence.groupby(['date', 'hour']).size().reset_index(name="count")

/tmp/ipykernel_15430/847945583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skip_occurrence['hour'] = skip_occurrence.index.hour
/tmp/ipykernel_15430/847945583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skip_occurrence['date'] = skip_occurrence.index.date


,date,hour,count
0,2023-01-01,15,2
1,2023-01-01,20,1
2,2023-01-03,9,1
3,2023-01-03,22,2
4,2023-01-08,2,5
...,...,...,...
353,2023-10-30,14,5
354,2023-10-30,22,1
355,2023-10-31,18,1
356,2023-11-01,19,3
